En primer lugar creo que es conveniente guardar los datos ya usados en el template inicial, para que no haya necesidad de calcularlos cada vez que se carga el notebook.

En segundo lugar, se propone este notebook como un espacio de experimentacion de creacion de features, para tener mejor ordenada cada tarea.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# This will allow you to see all column names & rows when you are doing .head(). None of the column name will be truncated.
# source: https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [4]:
# source: https://gist.github.com/rozanecm/38f2901c592bdffc40726cb0473318cf
# Function which plays a beep of given duration and frequency.
# Useful for when executing things that need a while to finish, to get notified.
import os
def beep(duration = 1, freq = 1500):
    """ play tone of duration in seconds and freq in Hz. """
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

In [5]:
train = pd.read_csv('../data/train.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])
test = pd.read_csv('../data/test.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])
sample_submission = pd.read_csv('../data/ejemploRespuesta.csv')

In [6]:
rescued_coords = pd.read_csv('../data/rescueLatLongs.csv')
precios_en_dolares = pd.read_csv('../data/precios_en_dolares.csv')

***

In [7]:
def contar_amenities(row):
    return row['gimnasio'] + row['usosmultiples'] + row['piscina'] + row['escuelascercanas'] + row['centroscomercialescercanos']

In [8]:
train['cant_amenities'] = train.apply(lambda x: contar_amenities(x), axis=1)
test['cant_amenities'] = test.apply(lambda x: contar_amenities(x), axis=1)

In [9]:
def feature_fechas(df):
    # Para entender lo de los senos y cosenos: https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
    df['year'] = df['fecha'].dt.year
    df['month'] = df['fecha'].dt.month
    df['day'] = df['fecha'].dt.day
    df['sin_month'] = np.sin(2*np.pi*df['month']/12)
    df['cos_month'] = np.cos(2*np.pi*df['month']/12)
    # tomo cant. de dias en mes: 31 en todos los casos. Para esto deberia servir bastante bien igual.
    df['sin_day'] = np.sin(2*np.pi*df['day']/31)
    df['cos_day'] = np.cos(2*np.pi*df['day']/31)
    
    # no necesito mas las cols. originales de month y day.
    df.drop(['month','day'], axis=1, inplace=True)
    
feature_fechas(train)
feature_fechas(test)

In [10]:
palabras_avenida = ['avenida', 'av']
train['es_avenida'] = train['direccion'].fillna('no info').apply(lambda x: any(avenida_indicator in x.lower() for avenida_indicator in palabras_avenida))
test['es_avenida'] = test['direccion'].fillna('no info').apply(lambda x: any(avenida_indicator in x.lower() for avenida_indicator in palabras_avenida))

In [21]:
train['titulo'].fillna("", inplace=True)

train['titulo_cant_html_tags'] = train['titulo'].apply(lambda x: len(x.split('<'))-1)

In [24]:
train['descripcion'].fillna("", inplace=True)

train['descripcion_cant_html_tags'] = train['descripcion'].apply(lambda x: len(x.split('<'))-1)

In [45]:
train.groupby('provincia')['antiguedad'].agg(['min','max','std','mean','median',len,'count','nunique'])

,min,max,std,mean,median,len,count,nunique
provincia,,,,,,,,
Aguascalientes,0.0,80.0,7.816406,7.523438,5.0,1753.0,1546,38
Baja California Norte,0.0,77.0,4.414062,5.789062,5.0,3220.0,2708,28
Baja California Sur,0.0,60.0,4.859375,2.458984,0.0,1700.0,1566,25
Campeche,0.0,35.0,6.500000,3.914062,0.0,263.0,199,20
Chiapas,0.0,80.0,7.531250,5.414062,4.0,1126.0,1010,32
Chihuahua,0.0,60.0,8.562500,6.519531,4.0,4592.0,2885,41
Coahuila,0.0,71.0,7.984375,6.304688,4.5,3696.0,3122,42
Colima,0.0,65.0,7.687500,5.675781,2.0,1008.0,812,32
Distrito Federal,0.0,80.0,11.820312,12.187500,10.0,58784.0,51749,76


In [26]:
train.sample(5)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,cant_amenities,year,sin_month,cos_month,sin_day,cos_day,es_avenida,titulo_cant_html_tags,descripcion_cant_html_tags
205002,196215,residencial wtc,"bonito departamento en piso 10, con excelente ...",Apartamento,DAKOTA,Benito Juárez,Distrito Federal,5.0,2.0,2.0,2.0,96.0,96.0,23568.0,NaN,NaN,2016-02-17,False,False,False,False,False,5000000.0,0,2016,0.866025,5.000000e-01,-0.299363,-0.954139,False,0,0
171738,260355,"casa en venta en hacienda de las palmas, huixq...",<p>preciosa casa ubicada en fraccionamiento co...,Casa,HACIENDA MARTIN CABALLERO,Huixquilucan,Edo. de México,3.0,4.0,NaN,4.0,NaN,270.0,55520.0,19.390625,-99.3750,2013-03-04,False,False,False,True,True,7000000.0,2,2013,1.000000,6.123234e-17,0.724793,0.688967,False,0,2
83295,78502,"casa en venta hacienda los morales, san nicolá...",hacienda los morales es una de las mejores col...,Casa,"Cebu 1304, Hacienda los Morales, San Nicolas",San Nicolás de los Garza,Nuevo León,20.0,4.0,1.0,3.0,135.0,105.0,inf,25.718750,-100.2500,2015-08-21,False,False,False,True,True,1150000.0,2,2015,-0.866025,-5.000000e-01,-0.897805,-0.440394,False,0,0
119719,269588,casa en venta en vista hermosa,"casa de 1 piso, dos recamaras con baño, amplia...",Casa,Rio Grijalva,Cuernavaca,Morelos,10.0,2.0,2.0,3.0,220.0,250.0,inf,18.937500,-99.2500,2012-04-16,False,False,False,True,True,1850000.0,2,2012,0.866025,-5.000000e-01,-0.101168,-0.994869,False,0,0
24166,60507,casa en venta 3 hab 2 1 /2 baños porto alegre ...,bien ubicada cerca de escuelas y plazas. cuent...,Casa,NaN,Cancún,Quintana Roo,NaN,NaN,NaN,2.0,90.0,76.0,inf,21.140625,-86.8125,2016-10-13,False,False,False,False,False,750000.0,0,2016,-0.866025,5.000000e-01,0.485302,-0.874347,False,0,0
